**[LeetCode Link](https://leetcode-cn.com/problems/find-k-th-smallest-pair-distance/solution/hei-ming-dan-zhong-de-sui-ji-shu-by-leetcode/)**

## 二分查找 + 双指针
### 方法
我们可以使用双指针来计算出所有小于等于 `mid` 的距离对数目。我们维护 `left` 和 `right`，其中 `right` 通过循环逐渐递增，`left` 在每次循环中被维护，使得它满足 `nums[right] - nums[left] <= guess` 且最小。这样对于 `nums[right]`，以它为右端的满足距离小于等于 `mid` 的距离对数目即为 `right - left`。我们在循环中对这些 `right - left` 进行累加，就得到了所有小于等于 `mid` 的距离对数目。

### 复杂度分析
* 时间复杂度：$\mathcal{O}(N \log W + N \log N)$，其中 $N$ 为 `nums` 数组的长度，$W$ 为 `nums` 数组中最大值与最小值的差，即 `nums[nums.length - 1] - nums[0]`（对 `nums` 数组进行排序之后）。其中二分查找的时间复杂度为 \log WlogW，计算 `possible(guess)` 函数的时间复杂度为 $\mathcal{O}(N)$，对 `nums` 数组进行排序的时间复杂度为 $\mathcal{O}(N\log N)$。
* 空间复杂度：$\mathcal{O}(1)$。

In [ ]:
class Solution:
    def smallestDistancePair(self, nums: list, k: int) -> int:
        # 二分搜索 + 双指针
        # 先将nums进行排序
        nums.sort()
        # 最大距离为首尾差
        # 第k小的距离差必然在 [min_distance, max_distance] 之间
        # 通过二分搜索确定距离差
        low, high = 0, nums[-1] - nums[0]
        while low < high:
            mid = (low + high) // 2
            # 淘汰策略
            # 对于mid而言
            # 若小于mid的距离差总数 >= k，则距离差应落在 [low, mid] 之间
            # 若大于mid的距离差总数 < k，则距离差应落在 [mid+1, high] 之间
            count = self.count_mid_distance(nums, mid)
            if count >= k:
                high = mid
            else:
                low = mid + 1
        return low

    def count_mid_distance(self, nums: list, target: int) -> int:
        # 由于数组已有序，所以我们只需要统计差值在target内的数量即可
        # 大于target的我们可以直接跳过，以此来减少计算次数
        # 依然使用动态窗口机制，我们每次计算至差值 <= target
        # 则窗口向右滑动时，两侧元素差值 > target，我们可以直接将左侧元素剔除
        left, count = 0, 0
        for right in range(1, len(nums)):
            # 每次将right与 [left, right] 中的每个元素进行比较
            # 由于数组有序，我们只需要将left移动至第一个满足 right-left <= tartget
            # 的位置即可，中间的元素即为满足条件的元素
            # 若无元素满足条件，则left追上right
            while nums[right] - nums[left] > target:
                left += 1
            count += right - left
        return count

## 堆（超时）
使用堆可以帮我们找到第 k 小值。我们将数组排序，此时对于固定的下标 i，从小到大的距离分别为 (i, i + 1)，(i, i + 2)，...，(i, N - 1)。因为 (i, j + 1) 的距离不会小于 (i, j)，因此如果 (i, j) 还在堆中，我们没有必要把 (i, j + 1) 放入堆中。

因此，我们首先将所有 (i, i + 1) 放入堆中，即 heap = [(i, i + 1) for all i]。每次取出堆中的最小元素 (i, j) 时（元素的大小为 nums[j] - nums[i]，即距离），再把 (i, j + 1) 放入堆中。直到取出 k 个元素，就得到了第 k 小的距离。

### 复杂度分析
* 时间复杂度：$\mathcal{O}((k + N) \log N)$，其中 $N$ 为 nums 数组的长度。因为 k 最大可以达到 $\mathcal{O}(N^2)$，因此最坏情况下，时间复杂度为 $\mathcal{O}(N^2 \log N)$，超出了时间限制。
* 空间复杂度：$\mathcal{O}(N)$。堆中的元素个数是 $\mathcal{O}(N)$ 的。

In [ ]:
class Solution(object):
    def smallestDistancePair(self, nums, k):
        nums.sort()
        heap = [(nums[i+1] - nums[i], i, i+1)
                for i in range(len(nums) - 1)]
        heapq.heapify(heap)

        for _ in range(k):
            d, root, nei = heapq.heappop(heap)
            if nei + 1 < len(nums):
                heapq.heappush(heap, (nums[nei + 1] - nums[root], root, nei + 1))

        return d